In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import seaborn as sns
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier

In [3]:
df = pd.read_csv('final.csv')

In [4]:
df = df.drop(['ARR_DELAY','Unnamed: 0'],1)
df

,QUARTER,MONTH,DAY_OF_MONTH,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,DEP_DEL15,CRS_ARR_TIME,ARR_DEL15,...,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity
0,1,1,1,13,6,745,-4.0,0.0,1602,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
1,1,1,1,10,6,1310,-4.0,0.0,1600,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
2,1,1,1,9,6,1330,38.0,1.0,1603,1.0,...,113,0.0,10,1015,0,32,26,41,32,74
3,1,1,1,8,6,800,-2.0,0.0,1624,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
4,1,1,1,11,6,1240,-4.0,0.0,1600,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
5,1,1,1,3,6,1150,-6.0,0.0,1620,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
6,1,1,1,1,6,1424,13.0,0.0,1611,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
7,1,1,1,8,6,730,-1.0,0.0,1554,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
8,1,1,1,9,6,1326,-1.0,0.0,1601,0.0,...,113,0.0,10,1015,0,32,26,41,32,74
9,1,1,1,8,6,735,1.0,0.0,1555,0.0,...,113,0.0,10,1015,0,32,26,41,32,74


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1802174 entries, 0 to 1802173
Data columns (total 22 columns):
QUARTER          int64
MONTH            int64
DAY_OF_MONTH     int64
ORIGIN           int64
DEST             int64
CRS_DEP_TIME     int64
DEP_DELAY        float64
DEP_DEL15        float64
CRS_ARR_TIME     int64
ARR_DEL15        float64
windspeedKmph    int64
winddirDegree    int64
weatherCode      int64
precipMM         float64
visibility       int64
pressure         int64
cloudcover       int64
DewPointF        int64
WindGustKmph     int64
tempF            int64
WindChillF       int64
humidity         int64
dtypes: float64(4), int64(18)
memory usage: 302.5 MB


In [135]:
df = df.astype(int)
#df = df.drop(['DEP_DELAY','DEP_DEL15'],1)

## Decision Tree Classifier

In [18]:
X = df.drop("ARR_DEL15", axis = 1)
Y = df["ARR_DEL15"]
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)

In [19]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [20]:
print("test score      : "+str(clf.score(x_test,y_test))) 
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score    : "+str(recall_score(y_test,y_pred))) 
print("f1 score        : "+str(f1_score(y_test,y_pred)))

test score      : 0.868134337675309
precision score : 0.6767788688088233
recall score    : 0.7058042236684818
f1 score        : 0.690986873329909


In [21]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.91      0.92    285145
         1.0       0.68      0.71      0.69     75290

    accuracy                           0.87    360435
   macro avg       0.80      0.81      0.80    360435
weighted avg       0.87      0.87      0.87    360435



## Sampling

### SMOTE

In [29]:
sm = SMOTE('auto', random_state=12, ratio=1.0)
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.30, random_state=42)
x_tr, y_tr = sm.fit_resample(x_train, y_train)

In [30]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_tr, y_tr)
y_pred = clf.predict(x_test)

In [31]:
print("Test Score: "+ str(clf.score(x_test,y_test))) 
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

Test Score: 0.8639719006460705
precision score : 0.6653109442241161
recall score : 0.6994482195766727
f1 score : 0.6819526371326264
              precision    recall  f1-score   support

         0.0       0.92      0.91      0.91    427927
         1.0       0.67      0.70      0.68    112726

    accuracy                           0.86    540653
   macro avg       0.79      0.80      0.80    540653
weighted avg       0.87      0.86      0.87    540653



### ADAYSN

In [10]:
sm = ADASYN()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [11]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

Test Score: 0.7285779682883183


In [12]:
print("Test Score: "+ str(clf.score(x_test,y_test))) 
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.3631161383600423
recall score : 0.39692152409790565
f1 score : 0.37926702367960213
              precision    recall  f1-score   support

         0.0       0.84      0.82      0.83    285138
         1.0       0.36      0.40      0.38     75297

    accuracy                           0.73    360435
   macro avg       0.60      0.61      0.60    360435
weighted avg       0.74      0.73      0.73    360435



### Random OverSampling

In [32]:
ros = RandomOverSampler(random_state=42)
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)

In [33]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

Test Score: 0.8708865676196818


In [34]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.6889134176949054
recall score : 0.6963341745251693
f1 score : 0.6926039196517626
              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92    285145
         1.0       0.69      0.70      0.69     75290

    accuracy                           0.87    360435
   macro avg       0.80      0.81      0.81    360435
weighted avg       0.87      0.87      0.87    360435



### Near Miss

In [16]:
nm = NearMiss()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = nm.fit_resample(x_train, y_train)

In [17]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

Test Score: 0.4714775202186247


In [18]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.23406111924208076
recall score : 0.6732804759817789
f1 score : 0.3473637329130837
              precision    recall  f1-score   support

         0.0       0.83      0.42      0.56    285138
         1.0       0.23      0.67      0.35     75297

    accuracy                           0.47    360435
   macro avg       0.53      0.55      0.45    360435
weighted avg       0.70      0.47      0.51    360435



### Random UnderSampling

In [12]:
rus = RandomUnderSampler()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = rus.fit_resample(x_train, y_train)

In [13]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

Test Score: 0.7917155659134102


In [14]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.5008977997699646
recall score : 0.8040111568601408
f1 score : 0.6172498355774672
              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86    285145
         1.0       0.50      0.80      0.62     75290

    accuracy                           0.79    360435
   macro avg       0.72      0.80      0.74    360435
weighted avg       0.85      0.79      0.81    360435



## Extra Trees Classifier 

In [10]:
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)

In [11]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.9084578356707866


In [12]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.8277513444818126
recall score : 0.709377075308806
f1 score : 0.7640062082925049
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94    285145
         1.0       0.83      0.71      0.76     75290

    accuracy                           0.91    360435
   macro avg       0.88      0.84      0.85    360435
weighted avg       0.91      0.91      0.91    360435



### Sampling

### SMOTE

In [13]:
sm = SMOTE()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)

In [14]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)
print("Test Score: "+ str(clf.score(x_test,y_test)))

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.905336607155243


In [15]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.7993485152546317
recall score : 0.7300836764510559
f1 score : 0.7631476647970233
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94    285145
         1.0       0.80      0.73      0.76     75290

    accuracy                           0.91    360435
   macro avg       0.86      0.84      0.85    360435
weighted avg       0.90      0.91      0.90    360435



### ADASYN

In [35]:
sm = ADASYN()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [36]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)
print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.9044404677681135
precision score : 0.7932093891321513
recall score : 0.7338424757603932
f1 score : 0.7623719341819312
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94    285145
         1.0       0.79      0.73      0.76     75290

    accuracy                           0.90    360435
   macro avg       0.86      0.84      0.85    360435
weighted avg       0.90      0.90      0.90    360435



### Random OverSampler

In [37]:
ros = RandomOverSampler(random_state=42)
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)

In [38]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)
print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.9070290066170045
precision score : 0.8273371149205553
recall score : 0.7012750697303759
f1 score : 0.7591080311700261
              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94    285145
         1.0       0.83      0.70      0.76     75290

    accuracy                           0.91    360435
   macro avg       0.88      0.83      0.85    360435
weighted avg       0.90      0.91      0.90    360435



### Near Miss

In [40]:
nm = NearMiss()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = nm.fit_resample(x_train, y_train)

In [41]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)
print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.7164398573945372
precision score : 0.41263122358486276
recall score : 0.844175853366981
f1 score : 0.5543146942495454
              precision    recall  f1-score   support

         0.0       0.94      0.68      0.79    285145
         1.0       0.41      0.84      0.55     75290

    accuracy                           0.72    360435
   macro avg       0.68      0.76      0.67    360435
weighted avg       0.83      0.72      0.74    360435



### Random UnderSampler

In [42]:
rus = RandomUnderSampler()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = rus.fit_resample(x_train, y_train)

In [43]:
clf = ExtraTreesClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)
print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

/home/rishi/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test Score: 0.8768266122879299
precision score : 0.6694010045401706
recall score : 0.8107318368973303
f1 score : 0.733318916840866
              precision    recall  f1-score   support

         0.0       0.95      0.89      0.92    285145
         1.0       0.67      0.81      0.73     75290

    accuracy                           0.88    360435
   macro avg       0.81      0.85      0.83    360435
weighted avg       0.89      0.88      0.88    360435



## Gradient Boosting

In [6]:
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)

In [7]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

Test Score: 0.9169031864275112


In [8]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.8949734297412667
recall score : 0.6822552795856024
f1 score : 0.7742698873271282
              precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    285145
         1.0       0.89      0.68      0.77     75290

    accuracy                           0.92    360435
   macro avg       0.91      0.83      0.86    360435
weighted avg       0.92      0.92      0.91    360435



## Sampling

### SMOTE

In [9]:
sm = SMOTE()
x_train, y_train = sm.fit_resample(x_train, y_train)

In [10]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test))) 

Test Score: 0.9122532495456879


In [11]:
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

precision score : 0.8291892217916434
recall score : 0.7303891619072918
f1 score : 0.7766596755855913
              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95    285145
         1.0       0.83      0.73      0.78     75290

    accuracy                           0.91    360435
   macro avg       0.88      0.85      0.86    360435
weighted avg       0.91      0.91      0.91    360435



### ADASYN

In [44]:
sm = ADASYN()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [45]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

Test Score: 0.9082358816430147
precision score : 0.800441249733115
recall score : 0.7469119404967459
f1 score : 0.7727506956611356
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94    285145
         1.0       0.80      0.75      0.77     75290

    accuracy                           0.91    360435
   macro avg       0.87      0.85      0.86    360435
weighted avg       0.91      0.91      0.91    360435



### Random Over Sampler

In [46]:
ros = RandomOverSampler(random_state=42)
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)

In [47]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

Test Score: 0.895015744863845
precision score : 0.7304048234280792
recall score : 0.7884181166157525
f1 score : 0.7583035258048032
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93    285145
         1.0       0.73      0.79      0.76     75290

    accuracy                           0.90    360435
   macro avg       0.84      0.86      0.85    360435
weighted avg       0.90      0.90      0.90    360435



### Near Miss

In [48]:
nm = NearMiss()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = nm.fit_resample(x_train, y_train)

In [49]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

Test Score: 0.7490476784995908
precision score : 0.4458438107212253
recall score : 0.8289414264842608
f1 score : 0.5798284976355714
              precision    recall  f1-score   support

         0.0       0.94      0.73      0.82    285145
         1.0       0.45      0.83      0.58     75290

    accuracy                           0.75    360435
   macro avg       0.69      0.78      0.70    360435
weighted avg       0.84      0.75      0.77    360435



### Random Under Sampler

In [50]:
rus = RandomUnderSampler()
X = np.array(df.drop("ARR_DEL15", axis = 1))
Y = np.array(df["ARR_DEL15"])
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.20, random_state=42)
x_train, y_train = rus.fit_resample(x_train, y_train)

In [51]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train) 
y_pred = clf.predict(x_test)

print("Test Score: "+ str(clf.score(x_test,y_test)))
print("precision score : "+str(precision_score(y_test,y_pred))) 
print("recall score : "+str(recall_score(y_test,y_pred))) 
print("f1 score : "+str(f1_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))

Test Score: 0.8947799187093374
precision score : 0.7295175616408064
recall score : 0.7887103200956302
f1 score : 0.7579600354843032
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93    285145
         1.0       0.73      0.79      0.76     75290

    accuracy                           0.89    360435
   macro avg       0.84      0.86      0.85    360435
weighted avg       0.90      0.89      0.90    360435

